In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import glob

In [3]:
#各種データフレーム呼び込み
rank_df = pd.read_csv("rank3_df.csv",index_col=0)
meta_df = pd.read_csv("meta3_df.csv",index_col=0)
emotion_df = pd.read_csv("emotion_df.csv",index_col=0)

In [5]:
#各動画の特徴語復元
movie_df = pd.read_csv("movie3_df_list.csv",index_col=0)
movie_df = movie_df.rename(columns={'id': 'ID'})
for i in range(len(movie_df)):
    movie_df["content"].iloc[i] = movie_df["content"].iloc[i].split(" ")
    movie_df["ID"].iloc[i] = movie_df["ID"].iloc[i].split("/")[1].split(".")[0]
    #",".join(movie_df["content"].iloc[0])

In [6]:
# 正規化
emotion_df["emo0"] = (emotion_df["emo0"] - emotion_df["emo0"].min()) / (emotion_df["emo0"].max() - emotion_df["emo0"].min())
emotion_df["emo1"] = (emotion_df["emo1"] - emotion_df["emo1"].min()) / (emotion_df["emo1"].max() - emotion_df["emo1"].min())
emotion_df["emo2"] = (emotion_df["emo2"] - emotion_df["emo2"].min()) / (emotion_df["emo2"].max() - emotion_df["emo2"].min())
emotion_df["emo3"] = (emotion_df["emo3"] - emotion_df["emo3"].min()) / (emotion_df["emo3"].max() - emotion_df["emo3"].min())

In [7]:
df = pd.merge(movie_df, emotion_df, left_on='ID', right_on='ID')

In [12]:
import gensim
from gensim.models import word2vec 
import logging
import sys
import gensim.downloader as gendl
import pyemd
from gensim.models import KeyedVectors
from gensim.test.utils import datapath
model = KeyedVectors.load_word2vec_format(
    datapath("/mnt/data1/home/ooga/models/jawiki.all_vectors.300d.txt"),
    binary=False
)

In [4]:
#距離，印象合わせて推薦
combrec_df=pd.DataFrame(columns=["ID", "comb_rec1","comb_rec2","comb_rec3"])
target = 0
rank = []
for x in range(len(df)):
    rec_df=pd.DataFrame(columns=["ID", "wmd_score", "emo_score"])
    target = x
    tar_emo = np.array([df["emo0"].iloc[x],df["emo1"].iloc[x],df["emo2"].iloc[x],df["emo3"].iloc[x]])
    a = 0
    for i in range(len(df)):
        if target != i:
            sim_value = model.wmdistance(df["content"].iloc[target], df["content"].iloc[i])
            i_emo = np.array([df["emo0"].iloc[i],df["emo1"].iloc[i],df["emo2"].iloc[i],df["emo3"].iloc[i]])
            emo_value = np.linalg.norm(tar_emo - i_emo)
            rec_df.loc[a] = [df["ID"].iloc[i],sim_value,emo_value]
            a += 1
            
    rec_df["wmd_score"] = (rec_df["wmd_score"] - rec_df["wmd_score"].min()) / (rec_df["wmd_score"].max() - rec_df["wmd_score"].min())
    rec_df["emo_score"] = (rec_df["emo_score"] - rec_df["emo_score"].min()) / (rec_df["emo_score"].max() - rec_df["emo_score"].min())
    rec_df["score"] = rec_df["wmd_score"] + rec_df["emo_score"]
    cal = rec_df.nsmallest(3, 'score')
    combrec_df.loc[x] = [df["ID"].iloc[x],cal["ID"].iloc[0],cal["ID"].iloc[1],cal["ID"].iloc[2]]
    if(x % 10 == 0):
        print(x)
combrec_df

NameError: name 'df' is not defined

In [59]:
#combrec_df.to_csv("combrec_st_df.csv")

In [ ]:
#印象のみで推薦
emorank_df=pd.DataFrame(columns=["ID","emo_rec1", "1_emo", "emo_rec2", "2_emo", "emo_rec3", "3_emo"])
target = 0
for x in range(len(emotion_df)):
    a = 0
    rec_df=pd.DataFrame(columns=["ID", "score"])
    target = x
    tar_emo = np.array([emotion_df["emo0"].iloc[x],emotion_df["emo1"].iloc[x],emotion_df["emo2"].iloc[x],emotion_df["emo3"].iloc[x]])
    for i in range(len(emotion_df)):
        if target != i:
            i_emo = np.array([emotion_df["emo0"].iloc[i],emotion_df["emo1"].iloc[i],emotion_df["emo2"].iloc[i],emotion_df["emo3"].iloc[i]])            
            emo_score = np.linalg.norm(tar_emo - i_emo)
            rec_df.loc[a] = [emotion_df["ID"].iloc[i], emo_score]
            a += 1
    cal = rec_df.nsmallest(3, 'score')
    emorank_df.loc[x] = [emotion_df["ID"].iloc[x],cal["ID"].iloc[0],cal["score"].iloc[0],cal["ID"].iloc[1],cal["score"].iloc[1],cal["ID"].iloc[2],cal["score"].iloc[2]]
    if(x % 10 == 0):
        print(x)
emorank_df

0
10


In [88]:
#emorank_df.to_csv("emorank_df.csv")

In [ ]:
#ここからreccomend_df作成

In [2]:
rank_df = pd.read_csv("rank3_df.csv",index_col=0)
meta_df = pd.read_csv("meta3_df.csv",index_col=0)
emotion_df = pd.read_csv("emotion_df.csv",index_col=0)
emorank_df = pd.read_csv("emorank_df.csv",index_col=0)
combrec_df = pd.read_csv("combrec_df.csv",index_col=0)

In [3]:
meta_df = meta_df.drop(columns=['watch_num', 'comment_num', 'mylist_num', 'description','upload_time', 'file_type', 'length', 'size_high','size_low', 'file'])

In [4]:
#ID入力 rank_df
for i in range(len(rank_df)):
    rank_df["1_num"].iloc[i] = rank_df["ID"].iloc[int(rank_df["1_num"].iloc[i])]
    rank_df["2_num"].iloc[i] = rank_df["ID"].iloc[int(rank_df["2_num"].iloc[i])]
    rank_df["3_num"].iloc[i] = rank_df["ID"].iloc[int(rank_df["3_num"].iloc[i])]

/usr/local/lib/python3.8/dist-packages/pandas/core/indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


In [5]:
movie_df = pd.read_csv("movie3_df_list.csv",index_col=0)
movie_df = movie_df.rename(columns={'id': 'ID'})
for i in range(len(movie_df)):
    movie_df["ID"].iloc[i] = movie_df["ID"].iloc[i].split("/")[1].split(".")[0]
    movie_df["content"].iloc[i] = movie_df["content"].iloc[i].split(" ")
    movie_df["content"].iloc[i] = ",".join(movie_df["content"].iloc[i])

rank_df = rank_df.drop(columns=['1_score','1_mark','2_score', '2_mark','3_score', '3_mark', 'imp', 'group1', 'group2', 'group3','group4', 'imp_check'])
rank_df = rank_df.rename(columns={'1_num': 'sim_rec1','2_num': 'sim_rec2','3_num': 'sim_rec3'})
reccomend_df = pd.merge(meta_df, movie_df, left_on='ID', right_on='ID')
reccomend_df = pd.merge(reccomend_df, rank_df, left_on='ID', right_on='ID')

In [6]:
emorank_df = emorank_df.drop(columns=['1_emo','2_emo','3_emo'])
emotion_df = emotion_df.drop(columns=["imp"])
emo = pd.merge(emotion_df, emorank_df, left_on='ID', right_on='ID')

In [7]:
reccomend_df = pd.merge(reccomend_df, emo, left_on='ID', right_on='ID')

In [8]:
reccomend_df = pd.merge(reccomend_df, combrec_df, left_on='ID', right_on='ID')

In [9]:
reccomend_df

,ID,title,category,tags,content,sim_rec1,sim_rec2,sim_rec3,emo0,emo1,emo2,emo3,emo_rec1,emo_rec2,emo_rec3,comb_rec1,comb_rec2,comb_rec3
0,sm17961470,【初音ミク】幸福な死を【オリジナル曲】,music_sound,DAM&AMP;JOY配信中 VOCALOID VOCALOID伝説入リ VOCALOID幻...,"きくお,幸福,鬱くしい,好き,地獄,天国,綺麗,歌詞,ミク,ダブステ,中毒,サビ,世界観,ダ...",sm10150980,sm16203010,sm19856000,0.295711,0.317458,0.221115,0.165715,sm33260180,sm31680050,sm19028880,sm10150980,sm16203010,sm5430176
1,sm12419220,ポケモン廃人だが、言わせてほしい。,game,†シンジ† アニメ ゲーム ポケモン廃人 モコウ レジェンド実況者 最後マデ見ルベキ 神回,"シンジ,もこう,ポケモン,厳選,先生,廃人,ゾロア,ポカブ,つり,マジレス,孵化,うそなき,...",sm9297140,sm32392150,sm22610180,0.700321,0.079647,0.124231,0.095801,sm17673790,sm9297140,sm7427060,sm9297140,sm22610180,sm8891090
2,sm5430176,絵本『人柱アリス』手描きPV 【VOCALOID】,music_sound,KAITO MEIKO VOCALOID VOCALOID-PV殿堂入リ ボカロ5兄弟 ミン...,"アリス,レン,不思議,番目,おk,リン,ミク,カイト,真っ赤,一時停止,メイコ,薔薇,kai...",sm5483070,sm15108950,sm7415020,0.381822,0.256409,0.221126,0.140643,sm31808640,sm30964600,sm3207770,sm5483070,sm7415020,sm17961470
3,so15281450,「アイドルマスター」オープニング,anime,2011年夏アニメ 765PRO_ALLSTARS READY!! アイドルマスター アイド...,"アイマス,ナンバーワン,うたプリ,美希,ここ,周年,公式,op,akb,ラブライブ,職人,み...",sm31791630,sm33086310,sm37601190,0.476855,0.332810,0.146399,0.043935,sm24432950,sm30786580,sm28104000,sm24326780,sm31791630,so23867610
4,sm2069100,θ波潜在能力開発（ヒーリング系作業用BGM),music_sound,Θ波 アニメ色ノナイ作業用BGM ニコニコ自習室 ヒーリング 作業用BGM 受験生支援BGM...,"勉強,テスト,明日,集中,休憩,数学,レポート,時間,今日,みんな,英語,あと,課題,徹夜,...",sm2406770,sm18393920,sm10889370,0.333538,0.131726,0.394259,0.140476,sm16881350,sm39379690,sm20593300,sm18393920,sm2406770,sm39379690
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
495,sm5408420,【MAD】ルルーシュと学校の怪談5,anime,アニメ コードギアス タバコ屋サンノ角ニイルノ ルルーシュ ルルーシュト学校ノ怪談 万能スイ...,"ルル,ルルーシュ,太平洋,ひで,ギアス,ローゼン,触手,スザク,みのりん,シスター,おまw,...",sm5664150,sm4767230,sm10879860,0.777805,0.047402,0.074806,0.099988,sm5664150,sm736760,sm17241370,sm5664150,sm4767230,sm14834740
496,sm10116060,【とある科学の超電磁砲】とある組曲の超電磁砲,anime,アニメ アレンジメドレーリンク コメガ嫁争奪戦 トアル科学ノ超電磁砲 トアル組曲ノ超電磁砲 ...,"美琴,黒子,俺の嫁,職人,木山,初春,レールガン,禁書,上条さん,そげぶ,原木,ゥゥン,りっ...",sm472410,sm334140,sm9510630,0.472418,0.200775,0.252062,0.074744,sm29662026,sm20065790,sm311320,sm9510630,sm334140,sm472410
497,sm20715461,進撃の巨人 4話 よう・・・5年ぶりだな・・・,anime,アニメ ラスボス 超大型巨人 進撃ノ巨人 進撃名場面セレクション,"サシャ,巨人,エレン,ベルトルト,ミーナ,フラグ,駆逐,ネタバレ,駆逐系男子,画伯,トーマス...",sm21075820,sm21224047,sm21213049,0.519637,0.095967,0.278611,0.105785,sm7289181,sm27957580,so21334230,sm21075820,sm8613232,nm8232118
498,sm3120540,【作業用BGM】コードギアス OP ED 挿入歌 フル詰め合わせ【修正Ver.】,anime,ALI_PROJECT FLOW アニメ コードギアス ニワン語 ロボソンFULL 作業妨害...,"一番,シャーリー,ユフィ,ルル,解読不能,モザイクカケラ,ルルーシュ,ギアス,ピカレスク,c...",sm3884025,sm5664150,sm10879860,0.278564,0.380806,0.253737,0.086893,sm17067980,sm8228650,sm33086310,sm16203010,sm27485100,sm34214810


In [10]:
reccomend_df.to_csv("reccomend_df.csv")

In [126]:
reccomend_df

,ID,title,category,tags,content,sim_rec1,sim_rec2,sim_rec3,emo0,emo1,emo2,emo3,emo_rec1,emo_rec2,emo_rec3,comb_rec1,comb_rec2,comb_rec3
0,sm17961470,【初音ミク】幸福な死を【オリジナル曲】,music_sound,DAM&AMP;JOY配信中 VOCALOID VOCALOID伝説入リ VOCALOID幻...,きくお 幸福 鬱くしい 好き 地獄 天国 綺麗 歌詞 ミク ダブステ 中毒 サビ 世界観 ダ...,sm10150980,sm16203010,sm19856000,0.299781,0.301481,0.225595,0.173142,sm31680050,sm33260180,sm5430176,sm5430176,sm16203010,sm10150980
1,sm12419220,ポケモン廃人だが、言わせてほしい。,game,†シンジ† アニメ ゲーム ポケモン廃人 モコウ レジェンド実況者 最後マデ見ルベキ 神回,シンジ もこう ポケモン 厳選 先生 廃人 ゾロア ポカブ つり マジレス 孵化 うそなき ...,sm9297140,sm32392150,sm22610180,0.502260,0.070483,0.336492,0.090765,sm7427060,sm17673790,sm9297140,sm9297140,sm22610180,sm10086900
2,sm5430176,絵本『人柱アリス』手描きPV 【VOCALOID】,music_sound,KAITO MEIKO VOCALOID VOCALOID-PV殿堂入リ ボカロ5兄弟 ミン...,アリス レン 不思議 番目 おk リン ミク カイト 真っ赤 一時停止 メイコ 薔薇 kai...,sm5483070,sm15108950,sm7415020,0.367159,0.266842,0.214295,0.151703,sm31808640,sm3207770,sm30964600,sm5483070,sm7415020,sm17961470
3,so15281450,「アイドルマスター」オープニング,anime,2011年夏アニメ 765PRO_ALLSTARS READY!! アイドルマスター アイド...,アイマス ナンバーワン うたプリ 美希 ここ 周年 公式 op akb ラブライブ 職人 み...,sm31791630,sm33086310,sm37601190,0.435901,0.388200,0.128982,0.046917,sm24432950,sm18502460,sm30786580,sm31791630,sm33086310,sm9510630
4,sm2069100,θ波潜在能力開発（ヒーリング系作業用BGM),music_sound,Θ波 アニメ色ノナイ作業用BGM ニコニコ自習室 ヒーリング 作業用BGM 受験生支援BGM...,勉強 テスト 明日 集中 休憩 数学 レポート 時間 今日 みんな 英語 あと 課題 徹夜 ...,sm2406770,sm18393920,sm10889370,0.354983,0.121498,0.380047,0.143472,sm39379690,sm16881350,sm25187377,sm18393920,sm2406770,sm39379690
5,so23867610,【試聴動画】ラブライブ！μ’s →NEXT LoveLive!2014 〜ENDLESS P...,music_sound,MUSIC_S.T.A.R.T!! Μ'S Μ'S_→NEXT_LOVELIVE!_2014...,くっすん えみつん うっちー りっぴー ジョルノ みもりん bibi ラブライブ 宮田 ライ...,so27026038,so20811229,sm22230060,0.414524,0.344303,0.181454,0.059718,so20811229,sm16814250,sm6580450,so27026038,so20811229,sm9874560
6,sm19817730,２分半でわかるジョジョの奇妙な冒険 第一部,anime,〇分デワカルジョジョシリーズ アニメ ジョジョMAD ジョジョ〜ソノ血ノ運命〜 ジョジョノ奇...,波紋 ジョジョ ディオ ジョナサン スピードワゴン ナイフ 石仮面 紳士 疾走 スト様 ナサ...,sm19650590,sm20540100,sm16024389,0.483674,0.071641,0.357437,0.087248,sm27721038,sm15878480,sm23510280,sm19650590,sm20540100,sm8157950
7,sm16601690,【Fate】全サーヴァント集 Part1,anime,FATE FATE/HOLLOW_ATARAXIA FATE/STAY_NIGHT FATE...,幸運 セイバー ギル マスター ランサー 士郎 魔力 エミヤ チート サーヴァント ライダー...,sm12294250,sm24644910,sm26325360,0.545455,0.074760,0.281599,0.098187,sm23261650,sm32392150,sm20569190,sm24644910,sm12294250,sm26325360
8,sm17241370,文学少女の日常（1～10話まとめ）,anime,アニメ デモ少シ・・・コノ風・・・泣イテイマス ヤッサン 今日ハ風ガ騒ガシイナ 文学少女(男...,日笠 クニ かお きた 上着 ポテト タダ いかん ゃひ ぇっ 杉田 半額 はふ 文學 ヒデ...,sm9063970,sm24106810,sm22001720,0.526966,0.051777,0.327513,0.093745,sm16620130,sm17629610,sm736760,sm9063970,sm13985710,sm736760
9,so27026038,【試聴動画】ラブライブ！μ’s Go→Go! LoveLive! 2015～Dream Se...,music_sound,アニソン ラブライブ! 音楽,えみつん うっちー 衣装 くっすん りっぴー コール みもりん ジョルノ ちゃん ライブ t...,so23867610,so20811229,sm24326780,0.369012,0.343763,0.229391,0.057833,sm31159377,sm18257340,sm30722500,so23867610,so20811229,sm31159377
